In [1]:
from sympy import *    

In [2]:
# symbol definition

t = symbols('t')
a = symbols('alpha')

display(t)
display(a)

t

alpha

In [3]:
# parametric function

fx = cos(t*a) + 1/3
fy = sin(t+1)

display(fx)
display(fy)

cos(alpha*t) + 0.333333333333333

sin(t + 1)

In [4]:
# first derivative

fx1 = fx.diff(t)
fy1 = fy.diff(t)

display(fx1)
display(fy1)

-alpha*sin(alpha*t)

cos(t + 1)

In [5]:
# second derivative

fx2 = fx1.diff(t)
fy2 = fy1.diff(t)

display(fx2)
display(fy2)

-alpha**2*cos(alpha*t)

-sin(t + 1)

In [6]:
import numpy as np

In [7]:
# evaluation & derivatives

def f0_parametric(t1, alpha1):

    return  np.array( [fx.subs({t:t1, a:alpha1}).evalf() , fy.subs({t:t1}).evalf() ],  dtype=np.float64 )

def f1_parametric(t1, alpha1):

    return np.array( [fx1.subs({t:t1, a:alpha1}).evalf() , fy1.subs({t:t1}).evalf() ] , dtype=np.float64)

def f2_parametric(t1, alpha1,):

    return np.array( [fx2.subs({t:t1, a:alpha1}).evalf() , fy2.subs({t:t1}).evalf() ], dtype=np.float64 )

In [8]:
import math

def curvature(t, a1):
      
        v = f1_parametric(t, a1)

        a = f2_parametric(t, a1)

        w = np.cross(v,a)

        magnitude = np.linalg.norm(w)

        speed = np.linalg.norm(v)

        return magnitude / math.pow(speed,3)

In [9]:
from IPython import display as disp

In [10]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import LinearSegmentedColormap    

In [11]:
cmap = plt.colormaps['viridis']

In [12]:
tetha = np.linspace(0, 2 * np.pi, 100)

In [13]:
from matplotlib.animation import FuncAnimation

# sm = plt.cm.ScalarMappable(cmap=cmap) 

def setup_fig_axs (obj_fun = np.array([])):

    fig, (axl, axr) = plt.subplots(
        
        ncols=2,
        figsize=(15, 5),
    )

    # creating ScalarMappable 
    sm = plt.cm.ScalarMappable(cmap=cmap) 
    
    fig.colorbar(sm, ax=axl) 

    # ax left

    lc_lft = LineCollection([], 
                            cmap=cmap, 
                            linewidth = 2.5)
    
    axl.add_collection(lc_lft)

    # ax right;
    ln_rgh, = axr.plot(tetha, tetha)
    axr.set_ylim([0, 80]) # fixed value;

    if (obj_fun.size > 0):

        axr.plot(tetha, obj_fun) # plotting t, b separately 

    axl.set_xlim(-1.5, 1.5)
    axl.set_ylim(-1.5, 1.5)

    return fig, lc_lft, sm, ln_rgh

In [14]:
# Function to update the line for each frame

def update(coeffs, lc_lft, sm, ln_rgh):

    t0_ = np.linspace(0, 2 * np.pi, 100)
    # fx_ = [f0_parametric.subs({t:t_})[0] for t_ in t0_]
    # fy_ = [f0_parametric.subs({t:t_})[1] for t_ in t0_]
    r0_ = np.array([f0_parametric(t_, coeffs[0]) for t_ in t0_])
    
    # render = np.array(list(zip(fx_, fy_))).reshape(-1,1,2)
    render   = r0_.reshape(-1,1,2)
    segments = np.concatenate([render[:-1],render[1:]], axis=1)
    curvatur = np.array([curvature(t2, coeffs[0]) for t2 in t0_])

    sm.set_array(curvatur.clip(0,1)) 

    lc_lft.set_segments(segments)
    lc_lft.set_array(curvatur.clip(0,1))
    lc_lft.cmap = cmap
    
    ln_rgh.set_ydata(curvatur)

    return ln_rgh, 

In [15]:
fig, lc, sm, line1 = setup_fig_axs()

ani = FuncAnimation(fig, 
                    update, 
                    fargs=[lc, sm, line1], 
                    frames= [np.array([a_]) 
                            for a_ in np.linspace(0, 1.0, 25)],
                    interval=200, 
                    blit=True) 

video = ani.to_html5_video() 
html = disp.HTML(video) 
  
disp.display(html) 
plt.close() 

In [16]:
def parametric_alpha_objective(alpha):

    # parametric stuff; 

    theta = np.linspace(0, 2 * np.pi, 100)

    curvt = np.array([curvature(t2, alpha) for t2 in theta])

    return ( theta, curvt )

In [17]:
def mse_gradient(x, y, vector):

    x1, y1 = parametric_alpha_objective(vector[0])

    return (2 / len(x1)) * np.sum(y - y1)

In [18]:
def gradient_descent(

    gradient, x, y,
    start,learn_rate=0.1, 
    n_iter=50, tolerance=1e-06,

    callback = lambda x : print(x)
):

    vector = start

    for _ in range(n_iter):

        diff = -learn_rate * np.array(gradient(x, y, vector))

        if np.all(np.abs(diff) <= tolerance):

            break

        vector += diff

        callback(np.copy(vector))

    return vector

In [19]:
x, y = parametric_alpha_objective(0.7)

values = [] 

gradient_descent(
    mse_gradient, x, y, start=[0.1], learn_rate=0.0001, n_iter=200,
    callback = lambda x : values.append(x)
)

print(values )

[array([0.1620131]), array([0.17141156]), array([0.17897356]), array([0.18538386]), array([0.19099186]), array([0.19600348]), array([0.20055165]), array([0.20472774]), array([0.20859758]), array([0.2122103]), array([0.21560366]), array([0.21880737]), array([0.22184525]), array([0.22473673]), array([0.22749788]), array([0.23014214]), array([0.23268092]), array([0.23512394]), array([0.23747958]), array([0.23975513]), array([0.24195697]), array([0.24409068]), array([0.24616122]), array([0.24817301]), array([0.25012999]), array([0.25203568]), array([0.25389328]), array([0.25570566]), array([0.25747545]), array([0.25920502]), array([0.26089656]), array([0.26255205]), array([0.26417332]), array([0.26576207]), array([0.26731985]), array([0.2688481]), array([0.27034816]), array([0.27182126]), array([0.27326857]), array([0.27469115]), array([0.27609002]), array([0.27746611]), array([0.27882031]), array([0.28015344]), array([0.28146629]), array([0.28275958]), array([0.28403401]), array([0.285290

In [20]:
cmap = plt.cm.get_cmap('jet')  

/tmp/ipykernel_13213/3779237694.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('jet')


In [21]:

fig, lc, sm, line1 = setup_fig_axs(obj_fun= y)

ani = FuncAnimation(fig, 
                    update, 
                    fargs=[lc, sm, line1], 
                    frames= values, 
                    interval = 200, 
                    blit=True)

video = ani.to_html5_video()
html = disp.HTML(video)

disp.display(html) 
plt.close() 